# Generate a signal on external trigger
This example showcases how to generate a burst signal on an exteral trigger. After the setup, Red Pitaya will wait for a trigger from an external source on DIO0_P (EXT_TRIG pin) and then generate the signal. The same concept also works for continuous signals.

## Libraries and FPGA image

In [ ]:
import time
from rp_overlay import overlay
import rp

fpga = overlay()
rp.rp_Init()

## Macros
Throughout this tutorial we will mention macros multiple times. Here is a complete list of macros that will come in handy when customising this notebook. The marcos are a part of the **rp** library.

- **Waveforms** - RP_WAVEFORM_SINE, RP_WAVEFORM_SQUARE, RP_WAVEFORM_TRIANGLE, RP_WAVEFORM_RAMP_UP, RP_WAVEFORM_RAMP_DOWN, RP_WAVEFORM_DC, RP_WAVEFORM_PWM, RP_WAVEFORM_ARBITRARY, RP_WAVEFORM_DC_NEG, RP_WAVEFORM_SWEEP
- **Generator modes** - RP_GEN_MODE_CONTINUOUS, RP_GEN_MODE_BURST
- **Sweep direction** - RP_GEN_SWEEP_DIR_NORMAL, RP_GEN_SWEEP_DIR_UP_DOWN
- **Sweep mode** - RP_GEN_SWEEP_MODE_LINEAR, RP_GEN_SWEEP_MODE_LOG
- **Generator trigger source** - RP_GEN_TRIG_SRC_INTERNAL, RP_GEN_TRIG_SRC_EXT_PE, RP_GEN_TRIG_SRC_EXT_NE
- **Fast analog channels** - RP_CH_1, RP_CH_2
- **Fast analog triggers** - RP_T_CH_1, RP_T_CH_2, RP_T_CH_EXT
- **Rise and fall times** - RISE_FALL_MIN_RATIO, RISE_FALL_MAX_RATIO

SIGNALlab 250-12 only:
- **Generator gain** - RP_GAIN_1X, RP_GAIN_5X

STEMlab 125-14 4-Input only:
- **Fast analog channels** - RP_CH_3, RP_CH_4
- **Fast analog triggers** - RP_T_CH_3, RP_T_CH_4

## External trigger
As we will see in a moment, the code is almost identical to the previous examples, except that we specify the trigger source as external.

Digital pin DIO0_P also acts as an External trigger pin. Red Pitaya assumes that the provided external signal is a 3V3 CMOS signal (pulse).

Additionally, a software debounce filter is present on the external trigger pin. The filter filters out unwanted bouncing on the external trigger transmission line and prevents potentially unwanted signal triggering. By default, the filter is set to 500 microseconds. If the external trigger signal is shorter or close to that value, we must change it.

Now that the theory is out of the way, it is time to do some coding.

In [ ]:
channel = rp.RP_CH_1        # rp.RP_CH_2
waveform = rp.RP_WAVEFORM_SINE
freq = 200
ampl = 1

mode = rp.RP_GEN_MODE_BURST

ncyc = 2            # Number of waveform periods in one burst
nor = 2             # Number of repeated bursts
period = 30000      # Delay between start of first burst and start of second burst
                    # in mircoseconds

gen_trig_sour = rp.RP_GEN_TRIG_SRC_EXT_PE   # Trigger source
debounce_len = 50000                        # debounce filter length in microseconds

# Reset generator
rp.rp_GenReset()

###### Generation #####
rp.rp_GenWaveform(channel, waveform)
rp.rp_GenFreqDirect(channel, freq)
rp.rp_GenAmp(channel, ampl)

# Change to burst mode
rp.rp_GenMode(channel, mode)
rp.rp_GenBurstCount(channel, ncyc)                  # Ncyc
rp.rp_GenBurstRepetitions(channel, nor)             # Nor
rp.rp_GenBurstPeriod(channel, period)               # Period

Everything so far should already be familiar. Here are the two lines that enable the external trigger and configure the debounce filter length.

In [ ]:
# Set length of internal debounce filter in us (minimum of 1 us)
rp.rp_GenSetExtTriggerDebouncerUs(debounce_len)

# Specify generator trigger source
rp.rp_GenTriggerSource(channel, gen_trig_sour)

The only thing left is enabling the output and wait for the external trigger.

In [ ]:
rp.rp_GenOutEnable(channel)

### Simulating an external trigger

**Recommendation:** To see the generated signal, use an oscilloscope or another Red Pitaya to measure the output signal. Alternatively, copy the code from one of the acquisition examples and connect OUT1 to IN1.

In case no external trigger source is available, we can simulate it with one of the Red Pitaya GPIO pins. Connect DIO0_N to DIO0_P (EXT_TRIG) with a jumper wire.

All that is left is configuring the DIO0_N pin as an output and changing its state.

In [ ]:
rp.rp_DpinReset()
rp.rp_DpinSetState(rp.RP_DIO0_N, rp.RP_LOW)
rp.rp_DpinSetDirection(rp.RP_DIO0_N, rp.RP_OUT)

Now, we can write a simple *for* loop to trigger the signal generation. There are a few things we must be aware of:
- The length of the external trigger pulse must be longer than the debounce filter length.
- The time between external trigger pulses must be longer than the generated signal.

The debounce filter will filter out signal pulses shorter than the specified time. For example, if the debounce filter is set to 500 microseconds and we apply a 10-microsecond pulse to the external trigger pin, it will not be detected.

The second problem is retriggering the signal before the generation is complete, which causes Red Pitaya to generate a combination of both signals, which can result in unexpected output signals.

Since our signal is relatively short, we don't need to be concerned with too fast retriggering, but we will still put some delay between triggers. Let us generate a simple square signal that will act as our external trigger.

In [ ]:
for i in range(0, 1):
    rp.rp_DpinSetState(rp.RP_DIO0_N, rp.RP_HIGH)        # External trigger - Positive edge
    time.sleep(0.2)
    rp.rp_DpinSetState(rp.RP_DIO0_N, rp.RP_LOW)         # External trigger - Negative edge
    time.sleep(0.2)

### Too short external trigger
Here, we simulate a too-short external trigger pulse that will not be detected due to the debounce filter.

Our debounce filter is currently set to 50 milliseconds. Let's see what happens if we apply a shorter signal to the External trigger pin (please take into consideration that the functions need some time to execute).

In [ ]:
for i in range(0, 1):
    rp.rp_DpinSetState(rp.RP_DIO0_N, rp.RP_HIGH)        # External trigger - Positive edge
    time.sleep(0.001)                                   # approx. 1 ms-long pulse
    rp.rp_DpinSetState(rp.RP_DIO0_N, rp.RP_LOW)         # External trigger - Negative edge
    time.sleep(0.001)

### Multiple external trigger pulses
Let us apply two consecutive external trigger pulses before the signal stops generating and see what happens. For this to work, we will crete a longer signal and shorten the debounce filter.

We will only modify the necessary settings, everything else is still the same as before. 

In [ ]:
ncyc = 2
nor = 1000
period = 20000      # Twice the lenght of one burst

debounce_len = 10   # microseconds

# Change to burst mode
rp.rp_GenBurstCount(channel, ncyc)                  # Ncyc
rp.rp_GenBurstRepetitions(channel, nor)             # Nor
rp.rp_GenBurstPeriod(channel, period)               # Period

rp.rp_GenSetExtTriggerDebouncerUs(debounce_len)     # setting the debounce to 10 us

for i in range(0,2):
    rp.rp_DpinSetState(rp.RP_DIO0_N, rp.RP_HIGH)     # External trigger - Positive edge
    time.sleep(0.001)
    rp.rp_DpinSetState(rp.RP_DIO0_N, rp.RP_LOW)      # External trigger - Negative edge
    time.sleep(0.005)
    # The second trigger is approx. 6 ms after the first one

Releasing the resources.

In [ ]:
rp.rp_Release()

### Note
There are a lot of different commands for the Generation. The list of available functions is quite an achievement to read through, so from now on, please refer to the *C and Python API section* of the [SCPI commands table](https://redpitaya.readthedocs.io/en/latest/appsFeatures/remoteControl/remoteControl.html#list-of-supported-scpi-commands) for all available commands.
